# Unit 5 — Functions, Modularity & Code Organization

**Purpose:** Transition from scripts to reusable code.

This unit teaches you how to:
- write **functions** to avoid repetition
- break larger problems into **small, testable pieces**
- understand **parameters**, **return values**, **default arguments**
- reason about **scope** (local vs. global)
- organize code into **modules** and use **imports**
- build basic **multi-file** Python programs

---

## Learning goals

By the end of this unit, you should be able to:

- Define and call functions
- Use parameters and return values correctly
- Choose between `print()` inside a function vs. returning values
- Use default arguments safely (and avoid mutable-default pitfalls)
- Understand local vs. global scope (and avoid common pitfalls)
- Refactor script-like code into functions
- Create and import your own module (multi-file program)

## 0) Why functions?

A function is a **named block of code** that performs a task.

Functions help you:
- **reuse** logic
- **reduce duplication**
- make code easier to **test**
- make programs easier to **read** and **maintain**

A good rule:
> If you copy/paste code more than once, consider a function.

## 1) Defining and calling functions

Basic syntax:

```python
def function_name(parameters):
    # body
    return result
```

- `def` introduces a function definition
- indentation defines the function body
- `return` sends a value back to the caller (and stops the function)

In [ ]:
def greet():
    print("Hello!")

greet()
greet()

### Functions that return values

A key distinction:

- `print(...)` shows output to the user
- `return ...` gives a value back to the program

Returning is usually more flexible.

In [ ]:
def add(a, b):
    return a + b

result = add(3, 4)
print("result:", result)
print("double:", result * 2)

### Multiple returns and early exit

`return` ends the function immediately.

In [ ]:
def classify_number(n):
    if n > 0:
        return "positive"
    if n == 0:
        return "zero"
    return "negative"

for n in [10, 0, -5]:
    print(n, "->", classify_number(n))

### `None` return value

If a function does not explicitly return something, it returns `None`.

In [ ]:
def say_hi(name):
    print(f"Hi {name}!")

x = say_hi("Ada")
print("Returned:", x)

## 2) Parameters, arguments, and return values

Terminology:
- **Parameter**: variable in the function definition
- **Argument**: value you pass when calling the function

Python supports:
- positional arguments
- keyword arguments

In [ ]:
def rectangle_area(width, height):
    return width * height

print(rectangle_area(3, 4))                 # positional
print(rectangle_area(width=3, height=4))    # keyword
print(rectangle_area(height=4, width=3))    # order doesn't matter with keywords

### Returning multiple values

A function can return multiple values by returning a tuple.

In [ ]:
def min_max(values):
    # simple implementation without using built-in min/max (for learning)
    if not values:
        return None, None

    mn = values[0]
    mx = values[0]
    for v in values[1:]:
        if v < mn:
            mn = v
        if v > mx:
            mx = v
    return mn, mx

mn, mx = min_max([3, 10, 7, 2, 8])
print("min:", mn, "max:", mx)

## 3) Default arguments

Default arguments allow you to make parameters optional. If the caller does not provide an argument for that parameter, the function uses the default value specified in the definition.

Key rules:
1. Default arguments must come **after** non-default (positional) arguments.
2. The default value expression is evaluated **once**, when the function is defined.

### Important caution: mutable default arguments

Avoid using mutable objects (like lists/dicts) as defaults.
We'll demonstrate the pitfall and the correct pattern.

In [ ]:
def greet2(name, greeting="Hello"):
    return f"{greeting}, {name}!"

print(greet2("Ada"))
print(greet2("Ada", greeting="Hi"))

### ⚠️ Common Pitfall: Mutable Default Arguments

You should **never** use a mutable object (like a `list` or `dict`) as a default argument.

**Why?**
Because default arguments are evaluated only **once** (at function definition time), not every time you call the function.
- If you use `container=[]`, that *exact same list object* is shared by every call that uses the default.
- If one call appends to it, the next call sees the modified list!

**Do not copy this pattern**:

In [ ]:
def add_item_bad(item, container=[]):
    container.append(item)
    return container

print(add_item_bad("a"))
print(add_item_bad("b"))  # surprise: container remembers previous call

### ✅ Correct pattern: Use `None`

The standard Pythonic way to handle this is to set the default value to `None`.
Inside the function, check for `None` and create a **new** list. This ensures a fresh empty list is created every time the function is called.

In [ ]:
def add_item(item, container=None):
    if container is None:
        container = []
    container.append(item)
    return container

print(add_item("a"))
print(add_item("b"))  # now it's a fresh list each call

## 4) Scope: local vs. global

- Variables created inside a function are **local**.
- Variables created outside are **global**.

A function can *read* global variables, but writing to them requires `global`.
In general: avoid `global` and prefer returning values.

In [ ]:
x = 10  # global

def show_x():
    print("inside show_x, x =", x)

show_x()
print("outside, x =", x)

### Local variables do not affect globals

In [ ]:
x = 10

def change_x_locally():
    x = 99  # local variable, does NOT change the global x
    return x

print("returned:", change_x_locally())
print("global x:", x)

### Writing to globals (usually a code smell)

This works, but is rarely the best design.

In [ ]:
counter = 0

def increment_global():
    global counter
    counter += 1

increment_global()
increment_global()
print("counter:", counter)

### Better approach: return updated values

In [ ]:
def increment(counter_value):
    return counter_value + 1

counter = 0
counter = increment(counter)
counter = increment(counter)
print("counter:", counter)

## 5) Code reuse and decomposition

Decomposition means breaking a larger problem into smaller functions.

Example: validating user input as an integer.

We’ll build:
1. A safe converter returning success/failure
2. A reusable input function that loops until valid

In [ ]:
def try_int(text):
    # Return (success, value)
    try:
        return True, int(text)
    except ValueError:
        return False, None

print(try_int("42"))
print(try_int("abc"))

### Reusable input helper

This function keeps asking until the user provides a valid integer.

Note: If `input()` is not available, a simulated iterator is used.

In [ ]:
_simulated = iter(["abc", "10"])

def safe_input(prompt: str) -> str:
    try:
        return input(prompt)
    except Exception:
        return next(_simulated)

def read_int(prompt: str, *, min_value=None, max_value=None):
    '''
    Read an integer from the user. Repeat until valid.
    Optional constraints: min_value, max_value.
    '''
    while True:
        text = safe_input(prompt).strip()
        ok, value = try_int(text)
        if not ok:
            print("Please enter a whole number.")
            continue
        if min_value is not None and value < min_value:
            print(f"Value must be >= {min_value}.")
            continue
        if max_value is not None and value > max_value:
            print(f"Value must be <= {max_value}.")
            continue
        return value

age = read_int("Age (0-120): ", min_value=0, max_value=120)
print("Age accepted:", age)

## 6) Refactoring earlier scripts into functions (example)

We refactor a menu-driven calculator into:
- `print_menu()`
- `get_choice()`
- `read_number()`
- `handle_choice()`
- `main()`

In [ ]:
_simulated_menu = iter(["1", "3", "4", "2", "5", "6", "3"])

def safe_input_menu(prompt: str) -> str:
    try:
        return input(prompt)
    except Exception:
        return next(_simulated_menu)

def try_float(text):
    try:
        return True, float(text)
    except ValueError:
        return False, None

def read_float(prompt: str):
    while True:
        text = safe_input_menu(prompt).strip()
        ok, value = try_float(text)
        if not ok:
            print("Please enter a valid number.")
            continue
        return value

def print_menu():
    print("\nMenu:")
    print("1) Add two numbers")
    print("2) Multiply two numbers")
    print("3) Quit")

def get_choice():
    while True:
        choice = safe_input_menu("Choose an option (1/2/3): ").strip()
        if choice in {"1", "2", "3"}:
            return choice
        print("Invalid choice. Try again.")

def handle_choice(choice: str):
    if choice == "3":
        print("Goodbye!")
        return False  # stop

    a = read_float("Enter first number: ")
    b = read_float("Enter second number: ")

    if choice == "1":
        print("Result:", a + b)
    elif choice == "2":
        print("Result:", a * b)

    return True

def main():
    while True:
        print_menu()
        choice = get_choice()
        if not handle_choice(choice):
            break

main()

## 7) Introduction to modules and imports (multi-file programs)

A **module** is a Python file (`.py`).  
When you `import` it, Python loads it and you can use its functions.

In this notebook we will create:
- `utils.py` — reusable helper functions
- `app.py` — a small program that imports `utils`

In Jupyter, we can write files using `%%writefile`.

### Create a utility module: `utils.py`

In [ ]:
%%writefile utils.py
def try_int(text):
    try:
        return True, int(text)
    except ValueError:
        return False, None


def read_int(prompt, *, min_value=None, max_value=None, input_func=input):
    '''
    Read an integer from input_func. Repeat until valid.
    '''
    while True:
        text = input_func(prompt).strip()
        ok, value = try_int(text)
        if not ok:
            print("Please enter a whole number.")
            continue
        if min_value is not None and value < min_value:
            print(f"Value must be >= {min_value}.")
            continue
        if max_value is not None and value > max_value:
            print(f"Value must be <= {max_value}.")
            continue
        return value


def mean(values):
    '''
    Compute the arithmetic mean of a non-empty list of numbers.
    '''
    if not values:
        raise ValueError("values must be non-empty")
    return sum(values) / len(values)


def word_frequency(text):
    '''
    Return a dict mapping words -> counts.
    Very simple tokenizer: lowercases and splits on whitespace.
    '''
    freq = {}
    for w in text.lower().split():
        freq[w] = freq.get(w, 0) + 1
    return freq

### Import and use the module

In [ ]:
import utils

print(utils.try_int("123"))
print(utils.try_int("abc"))

print("mean:", utils.mean([1, 2, 3, 4]))

freq = utils.word_frequency("Python is great and python is easy")
print(freq)

### Create an application file: `app.py`

In a terminal you would run:
```bash
python app.py
```

In a notebook, we can run it with `!python app.py`.

In [ ]:
%%writefile app.py
import utils

def main():
    print("Mini app: compute mean of n numbers")

    n = utils.read_int("How many numbers? (1-10): ", min_value=1, max_value=10)

    values = []
    for i in range(n):
        x = utils.read_int(f"Enter integer #{i+1}: ")
        values.append(x)

    print("Values:", values)
    print("Mean:", utils.mean(values))

if __name__ == "__main__":
    main()

In [ ]:
# Run the app (may prompt for input depending on environment)
!python app.py

---

# Practice (Exercises)

## Exercise 1 — Build a utility functions library

Create functions with docstrings:
1. `is_even(n) -> bool`
2. `clamp(x, min_value, max_value) -> number`
3. `count_vowels(text) -> int`
4. `normalize_name(name) -> str` (trim spaces + title case)

Then test them with a few examples.

In [ ]:
# Exercise 1 solution cell

def is_even(n):
    '''Return True if n is even, else False.'''
    return n % 2 == 0

def clamp(x, min_value, max_value):
    '''Clamp x into the inclusive range [min_value, max_value].'''
    if x < min_value:
        return min_value
    if x > max_value:
        return max_value
    return x

def count_vowels(text):
    '''Count vowels (a, e, i, o, u) in text (case-insensitive).'''
    vowels = set("aeiou")
    count = 0
    for ch in text.lower():
        if ch in vowels:
            count += 1
    return count

def normalize_name(name):
    '''Trim spaces and convert to title case.'''
    return " ".join(name.strip().split()).title()

print(is_even(10), is_even(7))
print(clamp(5, 0, 10), clamp(-1, 0, 10), clamp(99, 0, 10))
print(count_vowels("Hello World"))
print(normalize_name("  ada   lovelace  "))

## Exercise 2 — Refactor the number guessing game into functions

Suggested structure:
- `choose_secret(min_n, max_n)`
- `read_guess()` (with validation and optional quit)
- `evaluate_guess(guess, secret)` returning "low"/"high"/"correct"
- `game_loop()`
- `main()`

If `input()` is not available, use a simulated iterator.

In [ ]:
# Exercise 2 starter + reference solution

import random

_simulated_guesses = iter(["50", "75", "62", "q"])

def safe_input_guess(prompt: str) -> str:
    try:
        return input(prompt)
    except Exception:
        return next(_simulated_guesses)

def choose_secret(min_n=1, max_n=100):
    return random.randint(min_n, max_n)

def read_guess(prompt="Your guess (or q to quit): "):
    text = safe_input_guess(prompt).strip().lower()
    if text == "q":
        return None  # quit
    try:
        return int(text)
    except ValueError:
        print("Please enter an integer or 'q'.")
        return "invalid"

def evaluate_guess(guess, secret):
    if guess < secret:
        return "low"
    if guess > secret:
        return "high"
    return "correct"

def game_loop():
    secret = choose_secret(1, 100)
    attempts = 0
    print("Guess the number (1-100). Type 'q' to quit.")
    while True:
        guess = read_guess()
        if guess is None:
            print("You quit. Secret was:", secret)
            return
        if guess == "invalid":
            continue

        attempts += 1
        outcome = evaluate_guess(guess, secret)
        if outcome == "low":
            print("Too low")
        elif outcome == "high":
            print("Too high")
        else:
            print(f"Correct! Attempts: {attempts}")
            return

def main():
    game_loop()

main()

## Exercise 3 — Multi-file program (basic)

Create a module `text_utils.py` with:
- `word_frequency(text) -> dict`
- `top_n_words(freq_dict, n=5) -> list[tuple[word, count]]`

Then create `report.py` that:
- imports `text_utils`
- computes word frequencies for a given text
- prints the top N words

In [ ]:
%%writefile text_utils.py
def word_frequency(text):
    freq = {}
    for w in text.lower().split():
        freq[w] = freq.get(w, 0) + 1
    return freq

def top_n_words(freq_dict, n=5):
    items = sorted(freq_dict.items(), key=lambda kv: (-kv[1], kv[0]))
    return items[:n]

In [ ]:
%%writefile report.py
import text_utils

def main():
    text = "python is great and python is easy to learn and python is popular"
    freq = text_utils.word_frequency(text)
    top = text_utils.top_n_words(freq, n=5)

    print("Top words:")
    for word, count in top:
        print(f"{word}: {count}")

if __name__ == "__main__":
    main()

In [ ]:
!python report.py

---

## Checklist for Unit 5

You should now be comfortable with:

- [ ] Writing functions with `def`
- [ ] Using parameters and return values
- [ ] Using default arguments safely (avoid mutable defaults)
- [ ] Understanding local vs. global scope
- [ ] Decomposing a program into smaller functions
- [ ] Creating a module (`.py`) and importing it
- [ ] Building a simple multi-file program